In [ ]:
!pip install --quiet transformers torch scikit-learn
!pip install --quiet pandas numpy tqdm

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import pandas as pd
import json
import pickle
import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# Cấu hình và thiết lập

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
WINDOW_SIZE = 60  # Theo bài báo: long-term trend
EMBED_DIM = 48    # PCA: 768 → 48
BATCH_SIZE = 8
INPUT_JSON = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/X.json"
OUTPUT_NPZ = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/preprocessed_data.npz"
SCALERS_PKL = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/scalers.pkl"

print(f"[INFO] Thiết bị: {DEVICE}")
print(f"[INFO] Window size: {WINDOW_SIZE} ngày (theo bài báo gốc)")

[INFO] Thiết bị: cpu
[INFO] Window size: 60 ngày (theo bài báo gốc)


## phoBERT

In [ ]:
print("[INFO] Đang tải PhoBERT...")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
bert_model = AutoModel.from_pretrained("vinai/phobert-base").to(DEVICE)
bert_model.eval()

[INFO] Đang tải PhoBERT...


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
pca = PCA(n_components=EMBED_DIM)
news_scaler = StandardScaler()

In [ ]:
EVENT_KEYWORDS = ["hội nghị", "lễ hội", "APEC", "triển lãm", "đại hội", "du lịch", "hội thảo", "triển lãm", "festival"]


# Tính toán đặc trưng

In [ ]:
def detect_event_score(text):
    """Tăng độ nhạy với sự kiện (theo bài báo)"""
    if pd.isna(text) or not text.strip():
        return 0
    text = text.lower()
    score = sum(1 for kw in EVENT_KEYWORDS if kw in text)
    return min(score, 3)  # cap tại 3

def get_bert_embedding(text):
    """BERT embedding với xử lý missing"""
    if pd.isna(text) or not text.strip():
        return np.zeros(768)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True).to(DEVICE)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy().squeeze()

def load_data(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    rows = []
    for date, item in data.items():
        if "error" in item or "hotel" not in item:
            print(f"[WARNING] Thiếu 'hotel' tại {date} → Bỏ qua ngày này.")
            continue
        hotel = item["hotel"]
        env = item["env"]
        rows.append({
            "date": date,
            "single_price": hotel["single"]["price"],
            "single_available": hotel["single"]["available"],
            "single_sold": hotel["single"]["sold"],
            "double_price": hotel["double"]["price"],
            "double_available": hotel["double"]["available"],
            "double_sold": hotel["double"]["sold"],
            "vip_price": hotel["vip"]["price"],
            "vip_available": hotel["vip"]["available"],
            "vip_sold": hotel["vip"]["sold"],
            "revenue": hotel["revenue"],
            "is_weekend": env["is_weekend"],
            "is_holiday": env["is_holiday"],
            "holiday_score": env["holiday_score"],
            "local_event": env["local_event"],
            "event_score": env["event_score"],
            "weather_score": env["weather_score"],
            "comp_price_single": env["competitor_avg_single_price"],
            "comp_price_double": env["competitor_avg_double_price"],
            "comp_price_vip": env["competitor_avg_vip_price"],
            "news": env["news"]
        })
    df = pd.DataFrame(rows)
    df["date"] = pd.to_datetime(df["date"], format="%d-%m-%y")
    df = df.sort_values("date").reset_index(drop=True)
    print(f"[INFO] Đã tải {len(df)} ngày hợp lệ từ {df['date'].min()} đến {df['date'].max()}")
    return df

def add_derived_features(df):
    # Tính occupancy, free
    for room in ["single", "double", "vip"]:
        df[f"{room}_occupancy"] = df[f"{room}_sold"] / (df[f"{room}_available"] + 1e-8)
        df[f"{room}_free"] = df[f"{room}_available"] - df[f"{room}_sold"]
        df[f"{room}_revenue"] = df[f"{room}_sold"]* df[f"{room}_price"]
    # Event score từ news
    df["event_score_detected"] = df["news"].apply(detect_event_score)

    # Tổng event score
    df["event_score_final"] = df["event_score"] + df["event_score_detected"]

    return df, None

# Embeding



In [ ]:
def embed_news_and_pca(df):
    print("[INFO] Bắt đầu embedding news với BERT...")
    embeddings = []
    for i, text in enumerate(df["news"]):
        if i % 50 == 0:
            print(f"   Đang xử lý: {i}/{len(df)}")
        emb = get_bert_embedding(text)
        embeddings.append(emb)
    embeddings = np.array(embeddings)  # (N, 768)

    print(f"[INFO] Kích thước news embedding trước PCA: {embeddings.shape}")

    # PCA
    news_pca = pca.fit_transform(embeddings)

    # SCALE
    news_pca_scaled = news_scaler.fit_transform(news_pca)

    print(f"[INFO] Hoàn tất PCA + Scale → {news_pca_scaled.shape}")

    # Gán lại vào df
    for i in range(EMBED_DIM):
        df[f"news_emb_{i}"] = news_pca_scaled[:, i]

    return df

# TẠO CỬA SỔ TRƯỢT (SLIDING WINDOW)

In [ ]:
def create_sliding_windows(df, WINDOW_SIZE=60, EMBED_DIM=40):
    # ========================= SCALERS =========================
    price_scaler = MinMaxScaler()
    revenue_scaler = MinMaxScaler()


    # Fit scaler trên tất cả giá và doanh thu
    all_prices = np.concatenate([
        df['single_price'], df['double_price'], df['vip_price']
    ])
    all_revenues = np.concatenate([
        df['single_revenue'], df['double_revenue'], df['vip_revenue'], df['revenue']
    ])

    price_scaler.fit(all_prices.reshape(-1, 1))
    revenue_scaler.fit(all_revenues.reshape(-1, 1))

    scalers['demand'] = MinMaxScaler()
y_scaled = scalers['demand'].fit_transform(y_raw)

    # ========================= TẠO DỮ LIỆU =========================
    X_list = []      # (N, 60, 3, 24) – chỉ chứa đặc trưng phòng + env + revenue
    news_list = []   # (N, 60, 40) – tin tức truyền riêng
    y_list = []
    dates_list = []

    N = len(df)
    for i in range(WINDOW_SIZE, N):
        window = df.iloc[i - WINDOW_SIZE:i]
        target_date = df.iloc[i]["date"]

        # ---------- 1. Tạo X: (60, 3, 24) ----------
        X_window = np.zeros((WINDOW_SIZE, 3, 24), dtype=np.float32)
        news_window = np.zeros((WINDOW_SIZE, EMBED_DIM), dtype=np.float32)

        for t in range(WINDOW_SIZE):
            row = window.iloc[t]

            # News embedding – chỉ lưu riêng, KHÔNG gán vào X
            for emb_idx in range(EMBED_DIM):
                news_window[t, emb_idx] = row[f"news_emb_{emb_idx}"]

            for r, room in enumerate(["single", "double", "vip"]):
                base_idx = 0
                # 0: giá (chuẩn hóa)
                X_window[t, r, base_idx] = price_scaler.transform([[row[f"{room}_price"]]])[0, 0]
                # 1–4: trạng thái phòng
                X_window[t, r, base_idx+1] = row[f"{room}_available"]
                X_window[t, r, base_idx+2] = row[f"{room}_occupancy"]
                X_window[t, r, base_idx+3] = row[f"{room}_free"]
                X_window[t, r, base_idx+4] = row[f"comp_price_{room}"] if f"comp_price_{room}" in df.columns else 0
                # 5: DOANH THU RIÊNG của loại phòng (chuẩn hóa)
                X_window[t, r, base_idx+5] = revenue_scaler.transform([[row[f"{room}_revenue"]]])[0, 0]

            # 6: Tổng doanh thu ngày (chung cho 3 phòng)
            X_window[t, :, base_idx+6] = revenue_scaler.transform([[row["revenue"]]])[0, 0]

            # 7–11: Environment features (chung)
            env_feats = ['is_weekend', 'is_holiday', 'holiday_score', 'event_score_final', 'weather_score']
            for e_idx, e_feat in enumerate(env_feats):
                X_window[t, :, base_idx+7 + e_idx] = row[e_feat]

        # ---------- 2. Tạo y ----------
        target_row = df.iloc[i]
        y_sold = [
            target_row["single_sold"],
            target_row["double_sold"],
            target_row["vip_sold"]
        ]

        X_list.append(X_window)
        news_list.append(news_window)
        y_list.append(y_sold)
        dates_list.append(target_date)

    # ========================= CHUYỂN SANG NUMPY =========================
    X = np.array(X_list, dtype=np.float32)          # (N, 60, 3, 24)
    news_emb = np.array(news_list, dtype=np.float32) # (N, 60, 40)
    y = np.array(y_list, dtype=np.float32)          # (N, 3)

    print(f"[SUCCESS] Tạo dữ liệu hoàn tất!")
    print(f"     X.shape      = {X.shape}         ← d = 24")
    print(f"     news_emb.shape = {news_emb.shape} ← truyền riêng cho RPT")
    print(f"     y.shape      = {y.shape}")
    print(f"     Số mẫu       = {len(X)}")

    return X, news_emb, y, dates_list, {
        'price_scaler': price_scaler,
        'revenue_scaler': revenue_scaler
    }

# Main
Thể hiện quy trình embbed cũng như preprocessing của nhóm

In [ ]:
def main():
    print("=== BẮT ĐẦU GIAI ĐOẠN: NEWS EMBEDDING + PREPROCESS (CẢI THIỆN 2025) ===")

    # 1. Load dữ liệu gốc
    df = load_data(INPUT_JSON)

    # 2. Thêm các đặc trưng phái sinh (occupancy, free, revenue, v.v.)
    df, _ = add_derived_features(df)

    # 3. Embed tin tức → PCA → chuẩn hóa
    df = embed_news_and_pca(df)

    # 4. TẠO SLIDING WINDOW – PHIÊN BẢN TỐI ƯU NHẤT
    # → X: không chứa news_emb
    # → news_emb: truyền riêng
    # → Có thêm doanh thu (rất mạnh!)
    X, news_emb, y_demand, dates, scalers = create_sliding_windows(df)

    # 5. LƯU DỮ LIỆU – ĐÃ CẬP NHẬT ĐẦY ĐỦ
    np.savez_compressed(
        OUTPUT_NPZ,
        X=X,                          # (N, 60, 3, 28) – không có news_emb
        news_emb=news_emb,            # ← MỚI: lưu riêng tin tức
        y_demand=y_demand,            # (N, 3) – số phòng bán ra ngày mai
        dates=np.array(dates, dtype='datetime64[D]'),
        # Ghi chú cấu trúc đặc trưng trong X (d = 28)
        feature_cols_per_room=np.array([
            'price_scaled',           # 0
            'available',              # 1
            'occupancy',              # 2
            'free',                   # 3
            'comp_price',             # 4
            'room_revenue_scaled',    # 5 – MỚI: doanh thu từng loại phòng
            'total_revenue_scaled',   # 6 – MỚI: tổng doanh thu ngày
            'is_weekend',             # 7
            'is_holiday',             # 8
            'holiday_score',          # 9
            'event_score_final',      # 10
            'weather_score'           # 11
            # → Tổng: 12 đặc trưng/phòng × 3 + env chung = 28 chiều
        ])
    )

    # Lưu scaler
    with open(SCALERS_PKL, 'wb') as f:
        pickle.dump(scalers, f)

    # 6. IN THÔNG TIN HOÀN CHỈNH
    print(f"[SUCCESS] ĐÃ LƯU DỮ LIỆU TỐI ƯU 2025:")
    print(f"   → {OUTPUT_NPZ}")
    print(f"   → {SCALERS_PKL}")
    print(f"   - Số mẫu huấn luyện: {X.shape[0]:,}")
    print(f"   - X.shape         : {X.shape} ← d = 28 (đã loại news_emb)")
    print(f"   - news_emb.shape  : {news_emb.shape} ← truyền riêng cho RPT")
    print(f"   - y_demand.shape  : {y_demand.shape} ← [single, double, vip]_sold")
    print(f"   - Có thêm doanh thu → mô hình hiểu price elasticity!")
    print("=== HOÀN TẤT – THEO ĐÚNG BÀI BÁO GỐC (Mei, 2025) + CẢI TIẾN MẠNH ===")

    return X, news_emb, y_demand, dates  # ← Trả về đủ cả news_emb

# 6. HÀM CHÍNH

In [ ]:
if __name__ == "__main__":
    # ← PHIÊN BẢN MỚI: main() trả về 4 giá trị
    X, news_emb, y_demand, dates = main()

    # ============================= DEBUG SIÊU ĐẸP =============================
    print("\n" + "="*70)
    print("         DEBUG – DỰ ĐOÁN DEMAND THEO BÀI BÁO MEI (2025) + CẢI TIẾN")
    print("="*70)
    print(f"   Ngày dự đoán (ngày thứ 61): {dates[0]}")
    print(f"   Số phòng thực tế đã bán ra (ground truth):")
    print(f"     • Single : {int(y_demand[0, 0]):3d} phòng")
    print(f"     • Double : {int(y_demand[0, 1]):3d} phòng")
    print(f"     • VIP    : {int(y_demand[0, 2]):3d} phòng")
    print(f"   Tổng cộng    : {int(y_demand[0].sum()):3d} phòng")
    print("-"*70)
    print(f"   Cấu trúc dữ liệu:")
    print(f"     • X.shape        : {X.shape}         ← (N, 60, 3, 28) – không có news_emb")
    print(f"     • news_emb.shape : {news_emb.shape}  ← (N, 60, 40) – truyền riêng")
    print(f"     • y_demand.shape : {y_demand.shape}  ← (N, 3)")
    print("-"*70)
    print(f"   Mẫu dữ liệu ngày cuối cùng (ngày 60/60):")
    print(f"     • Giá Single  (scaled)     : {X[0, -1, 0, 0]:.4f}")
    print(f"     • Doanh thu Single (scaled): {X[0, -1, 0, 5]:.4f}")
    print(f"     • Tổng doanh thu ngày      : {X[0, -1, 0, 6]:.4f}")
    print(f"     • Là cuối tuần?            : {'Có' if X[0, -1, 0, 7] > 0.5 else 'Không'}")
    print(f"     • News embedding (3 dims đầu): {news_emb[0, -1, :3]} ... (40 dims)")
    print("-"*70)
    print("   DỮ LIỆU HOÀN HẢO – SẴN SÀNG HUẤN LUYỆN RPT + PPO!")
    print("="*70)

=== BẮT ĐẦU GIAI ĐOẠN: NEWS EMBEDDING + PREPROCESS (CẢI THIỆN 2025) ===
[WARNING] Thiếu 'hotel' tại 09-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 12-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 14-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 24-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 27-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 31-01-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 13-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 22-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 25-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 27-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 28-02-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 01-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 02-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 07-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 08-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 09-03-24 → Bỏ qua ngày này.
[WARNING] Thiếu 'hotel' tại 10-0